In [ ]:
import altair as alt
import pandas as pd
import yfinance as yf

In [ ]:
tickers_of_interest = ["AAPL", "AMZN", "GOOGL", "MSFT", "TSLA"]

In [ ]:
tickers = yf.Tickers(tickers_of_interest)

In [ ]:
hist = tickers.history()

In [ ]:
hist.tail()

In [ ]:
hist_df = hist["Close"]

In [ ]:
hist_df = hist_df.reset_index()

In [ ]:
base = alt.Chart(hist_df).mark_line().encode(
    x="Date:T",
    y="AAPL:Q",
)

In [ ]:
base